# Code Converter - Python to TypeScript Code

This implementation, converts python code to optimized TypeScript Code, and runs the function

## Set up and imports


In [115]:

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display, display_markdown
from system_info import retrieve_system_info
import gradio as gr

# Initializing the access keys

In [116]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set. Check your engironment variables and try again")

OpenAI API Key exists and begins sk-proj-


# Connecting to client libraries

In [117]:
openai = OpenAI()

In [118]:
# contants
OPENAI_MODEL= "gpt-5-nano"

In [119]:
system_info = retrieve_system_info()
system_info

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '24.5.0',
  'version': 'Darwin Kernel Version 24.5.0: Tue Apr 22 19:48:46 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T8103',
  'kernel': '24.5.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'arm64-apple-darwin24.5.0'},
 'package_managers': ['xcode-select (CLT)', 'brew'],
 'cpu': {'brand': 'Apple M1',
  'cores_logical': 8,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'Apple clang version 17.0.0 (clang-1700.0.13.3)',
   'g++': 'Apple clang version 17.0.0 (clang-1700.0.13.3)',
   'clang': 'Apple clang version 17.0.0 (clang-1700.0.13.3)',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

In [120]:
message = f"""
Here is a report of the system information for my computer.
I want to run a TypeScript compiler to compile a single TypeScript file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any TypeScript compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile TypeScript code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

In [121]:
response = openai.chat.completions.create(model=OPENAI_MODEL, messages=[{"role":"user", "content":message}])
display(Markdown(response.choices[0].message.content))

Short answer:
- Yes, to compile TypeScript you need a TypeScript compiler (tsc). On macOS you’ll typically install Node.js first, then install TypeScript.
- Important: main.cpp sounds like a C++ file. The TypeScript compiler (tsc) cannot compile .cpp. If you want to use TypeScript, rename the file to main.ts (and ensure its contents are TypeScript). If you actually meant C++, use a C++ compiler instead (clang/g++).

Step-by-step to set up TypeScript (simplest path on your system):
1) Install Node.js (which also installs npm)
- brew update
- brew install node

2) Install the TypeScript compiler globally
- npm install -g typescript

3) Verify installations
- node -v
- npm -v
- tsc -v

4) Compile and run a TypeScript file (assuming your file is main.ts)
- tsc main.ts
- node main.js

Notes:
- If your file is indeed C++ (main.cpp), you cannot compile it with tsc. To compile C++, use clang++ (on macOS) or g++:
  - clang++ -std=c++17 main.cpp -o main
  - ./main

Python integration (fill-in for your example)
- If you have a TypeScript file named main.ts and you want to compile it to JavaScript and then run it with Node, use:
  compile_command = ["tsc", "main.ts"]
  run_command = ["node", "main.js"]

- If you want to show a single command in Python that compiles and runs in one go (still two steps because TS compiles to JS first):
  compile_command = ["tsc", "main.ts"]
  run_command = ["node", "main.js"]

- If you truly want to bypass TypeScript and run C++ instead (not TypeScript):
  compile_command = ["clang++", "-std=c++17", "main.cpp", "-o", "main"]
  run_command = ["./main"]

If you’d like, tell me whether main.cpp is meant to be C++ or you actually have a TypeScript file named main.ts, and I can tailor the exact commands.

In [122]:
compile_command = ["tsc", "main.ts", "--target", "ES2020", "--module", "commonjs"]
run_command = ["ts-node", "main.ts"]

## System and user prompts for the code converter

In [123]:
system_prompt = """
Your task is to convert Python code into high performance TypeScript code.
Respond only with TypeScript code. Do not provide any explanation other than occasional comments.
The TypeScript response needs to produce an identical output in the fastest possible time.
"""


def user_prompt_for(python):
    return f""" 
        port this Python code to TypeScript with the fastest possible implementation that produces identical output in the least time.

        The system information is 

        {system_info}

        Your response will be written to a file called main.ts and then compile and ecexted; the compilation command is:

        {compile_command}

        Respond only with C++ code.
        Python code to port:

        ```python
        {python}
        ```

"""

In [124]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)},
    ]

In [125]:
def write_output(code):
    with open("main.ts", "w", encoding="utf-8") as f:
        f.write(code)

In [126]:
def convert(python):
    reasoning_effort = "high"
    response = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for(python),
        reasoning_effort=reasoning_effort,
    )
    reply = response.choices[0].message.content
    reply = reply.replace("```ts", "").replace("```", "")
    return reply

In [127]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [128]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [129]:
run_python(pi)

'Result: 3.141592656089\nExecution Time: 19.478347 seconds\n'

In [130]:
convert(pi)

"import { performance } from 'perf_hooks';\n\nfunction digamma(z: number): number {\n  let acc = 0;\n  while (z < 7) {\n    acc -= 1 / z;\n    z += 1;\n  }\n  const z2 = z * z;\n  const z4 = z2 * z2;\n  const z6 = z4 * z2;\n  const z8 = z4 * z4;\n  const z10 = z8 * z2;\n  const z12 = z10 * z2;\n  const series =\n    Math.log(z)\n    - 1 / (2 * z)\n    - 1 / (12 * z2)\n    + 1 / (120 * z4)\n    - 1 / (252 * z6)\n    + 1 / (240 * z8)\n    - 5 / (660 * z10)\n    + 691 / (32760 * z12);\n  return acc + series;\n}\n\nconst N = 200_000_000;\n\nconst t0 = performance.now();\nconst result =\n  4 - digamma(N + 0.75) + digamma(0.75) + digamma(N + 1.25) - digamma(1.25);\nconst t1 = performance.now();\n\nconsole.log(`Result: ${result.toFixed(12)}`);\nconsole.log(`Execution Time: ${((t1 - t0) / 1000).toFixed(6)} seconds`);"

In [131]:
        
def run_typescript(code):
    write_output(code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [132]:
# run_typescript()

## User Interface

In [133]:
with gr.Blocks(
    theme=gr.themes.Monochrome(), title="Port from Python to TypeScript"
) as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python Original Code",
                value=pi,
                language="python",
                lines=30,
            )
        with gr.Column(scale=6):
            ts = gr.Code(
                label="TypeScript (generated)", value="", language="cpp", lines=26
            )
    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("Run Python", elem_classes=["run-btn", "py"])
        port = gr.Button("Convert to TS", elem_classes=["convert-btn"])
        ts_run = gr.Button("Run TS", elem_classes=["run-btn", "ts"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python Result", lines=10)
        with gr.Column(scale=6):
            ts_out = gr.TextArea(label="TS output", lines=10)

    port.click(fn=convert, inputs=[python], outputs=[ts])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    ts_run.click(fn=run_typescript, inputs=[ts], outputs=[ts_out])
    
    
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
